Imports

In [2]:
import pandas as pd

In [ ]:
Data Loading

In [10]:
player_stats = pd.read_csv('/Users/martaperez/Desktop/Master ie/Term 3/Sports Analytics/atp_matches_2023.csv')

In [11]:
player_stats

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2023-9900,United Cup,Hard,18,A,20230102,300,126203,3.0,NaN,...,62.0,47.0,15.0,12.0,9.0,9.0,9.0,3355.0,16.0,2375.0
1,2023-9900,United Cup,Hard,18,A,20230102,299,126207,NaN,NaN,...,12.0,8.0,3.0,4.0,1.0,3.0,19.0,2000.0,23.0,1865.0
2,2023-9900,United Cup,Hard,18,A,20230102,296,126203,3.0,NaN,...,62.0,51.0,7.0,12.0,2.0,2.0,9.0,3355.0,10.0,2905.0
3,2023-9900,United Cup,Hard,18,A,20230102,295,126207,NaN,NaN,...,41.0,26.0,12.0,9.0,6.0,9.0,19.0,2000.0,245.0,220.0
4,2023-9900,United Cup,Hard,18,A,20230102,292,126774,1.0,NaN,...,58.0,48.0,18.0,16.0,1.0,2.0,4.0,5550.0,16.0,2375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,2023-M-DC-2023-WG2-PO-RSA-LUX-01,Davis Cup WG2 PO: RSA vs LUX,NaN,4,D,20230204,5,202335,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1717.0,1.0
2982,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,1,117365,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,990.0,11.0,279.0,190.0
2983,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,2,121411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,364.0,131.0,894.0,15.0
2984,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,4,144949,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,894.0,15.0,285.0,184.0


Player Info Dataset

In [14]:
import pandas as pd

# Load data
df = pd.read_csv("atp_matches_2023.csv")

# Create winner DataFrame
winners = df[['winner_name', 'surface', 'w_ace']].copy()
winners.columns = ['player', 'surface', 'aces']
winners['win'] = 1
winners['loss'] = 0

# Create loser DataFrame
losers = df[['loser_name', 'surface', 'l_ace']].copy()
losers.columns = ['player', 'surface', 'aces']
losers['win'] = 0
losers['loss'] = 1

# Combine into a single DataFrame
players_df = pd.concat([winners, losers], ignore_index=True)

# Ensure surface column is clean (sometimes spaces or NaNs)
players_df['surface'] = players_df['surface'].astype(str).str.strip()

# Group by player only
overall_stats = players_df.groupby('player').agg(
    matches=('win', 'count'),
    wins=('win', 'sum'),
    losses=('loss', 'sum'),
    total_aces=('aces', 'sum'),
    avg_aces=('aces', 'mean')
).reset_index()

overall_stats['win_pct'] = overall_stats['wins'] / overall_stats['matches']

# Optional: group by player AND surface
surface_stats = players_df.groupby(['player', 'surface']).agg(
    matches=('win', 'count'),
    wins=('win', 'sum'),
    win_pct=('win', 'mean')
).reset_index()

# Pivot surface-specific win % into columns
surface_pivot = surface_stats.pivot(index='player', columns='surface', values='win_pct').reset_index()
surface_pivot.columns.name = None  # remove pandas column grouping

# Merge overall and surface-specific stats
final_df = pd.merge(overall_stats, surface_pivot, on='player', how='left')

# Show result
print(final_df.head())


                 player  matches  wins  losses  total_aces  avg_aces  \
0    Abedallah Shelbayh       11     3       8        51.0  4.636364   
1          Adam Moundir        2     0       2         1.0  1.000000   
2      Adrian Mannarino       67    43      24       330.0  5.000000   
3  Aisam Ul Haq Qureshi        1     1       0         0.0       NaN   
4          Albert Ramos       38    12      26       112.0  2.947368   

    win_pct      Clay  Grass      Hard  nan  
0  0.272727  0.333333   0.00  0.333333  NaN  
1  0.000000  0.000000    NaN  0.000000  NaN  
2  0.641791  0.200000   0.75  0.652174  NaN  
3  1.000000       NaN   1.00       NaN  NaN  
4  0.315789  0.440000   0.00  0.090909  NaN  
